In [101]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
# import statsmodels.api as sm
%matplotlib inline 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn import metrics
from sklearn.model_selection import train_test_split 
from sklearn.decomposition import PCA

from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors

import requests
import kdtree as KDTree


In [102]:
df = pd.read_csv('houses_lat_long_v4.csv')

In [103]:
df = df.dropna(subset=['LAT_LONG'])

In [104]:
df

,PID,ST_NUM,ST_NAME,ST_NAME_SUF,ZIPCODE,OWN_OCC,AV_LAND,AV_BLDG,AV_TOTAL,GROSS_TAX,...,BTH_STYLE2,BTH_STYLE3,KITCH_STYLE,HEAT_TYP,AC,FPLACE,INT_CND,INT_FIN,VIEW,LAT_LONG
0,502555000,2,BEAVER,ST,2108,Y,1436300,1242500,2678800,2823455,...,NaN,NaN,M,W,N,0.0,G,N,A,"-71.07207,42.355618"
1,100001000,104 A 104,PUTNAM,ST,2128,Y,129900,411300,541200,570425,...,S,S,S,W,N,0.0,A,N,A,"-71.03256,42.379173"
2,100002000,197,LEXINGTON,ST,2128,N,121000,475100,596100,628289,...,M,M,M,F,C,0.0,A,N,A,"-71.03255,42.37957"
3,100003000,199,LEXINGTON,ST,2128,N,121400,421100,542500,571795,...,M,M,S,S,N,0.0,A,N,A,"-71.03248,42.379593"
4,100004000,201,LEXINGTON,ST,2128,N,121900,396400,518300,546288,...,S,S,S,W,N,0.0,A,N,A,"-71.03242,42.379616"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101662,1000319288,80-82,FENWOOD,RD,2115,Y,0,561500,561500,591821,...,NaN,NaN,M,F,C,0.0,G,N,A,"-71.10889,42.335903"
101663,1000319290,80-82,FENWOOD,RD,2115,Y,0,195300,195300,205847,...,NaN,NaN,M,F,C,0.0,G,N,A,"-71.10889,42.335903"
103067,1100253022,126,THORNTON,ST,2119,N,0,563900,563900,594351,...,M,NaN,M,F,C,0.0,G,N,A,"-71.09162,42.322926"
103068,1100253024,126,THORNTON,ST,2119,Y,0,614800,614800,647999,...,M,NaN,M,F,C,0.0,G,N,A,"-71.09162,42.322926"


In [105]:
# First, look for null values so we can dropna
null_columns=df.columns[df.isnull().any()]
df[null_columns].isnull().sum()

LAND_SF          19
YR_REMOD       7329
TOTAL_RMS        33
BTH_STYLE2    23961
BTH_STYLE3    58492
dtype: int64

In [106]:
df = df.fillna(0)

In [107]:
# # Get the coords
split_coords_lambda = lambda col: (col.split(",")[0], col.split(",")[1])
df['LAT'], df['LONG'] = zip(*df['LAT_LONG'].apply(split_coords))

In [108]:
df

,PID,ST_NUM,ST_NAME,ST_NAME_SUF,ZIPCODE,OWN_OCC,AV_LAND,AV_BLDG,AV_TOTAL,GROSS_TAX,...,KITCH_STYLE,HEAT_TYP,AC,FPLACE,INT_CND,INT_FIN,VIEW,LAT_LONG,LAT,LONG
0,502555000,2,BEAVER,ST,2108,Y,1436300,1242500,2678800,2823455,...,M,W,N,0.0,G,N,A,"-71.07207,42.355618",-71.07207,42.355618
1,100001000,104 A 104,PUTNAM,ST,2128,Y,129900,411300,541200,570425,...,S,W,N,0.0,A,N,A,"-71.03256,42.379173",-71.03256,42.379173
2,100002000,197,LEXINGTON,ST,2128,N,121000,475100,596100,628289,...,M,F,C,0.0,A,N,A,"-71.03255,42.37957",-71.03255,42.37957
3,100003000,199,LEXINGTON,ST,2128,N,121400,421100,542500,571795,...,S,S,N,0.0,A,N,A,"-71.03248,42.379593",-71.03248,42.379593
4,100004000,201,LEXINGTON,ST,2128,N,121900,396400,518300,546288,...,S,W,N,0.0,A,N,A,"-71.03242,42.379616",-71.03242,42.379616
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101662,1000319288,80-82,FENWOOD,RD,2115,Y,0,561500,561500,591821,...,M,F,C,0.0,G,N,A,"-71.10889,42.335903",-71.10889,42.335903
101663,1000319290,80-82,FENWOOD,RD,2115,Y,0,195300,195300,205847,...,M,F,C,0.0,G,N,A,"-71.10889,42.335903",-71.10889,42.335903
103067,1100253022,126,THORNTON,ST,2119,N,0,563900,563900,594351,...,M,F,C,0.0,G,N,A,"-71.09162,42.322926",-71.09162,42.322926
103068,1100253024,126,THORNTON,ST,2119,Y,0,614800,614800,647999,...,M,F,C,0.0,G,N,A,"-71.09162,42.322926",-71.09162,42.322926


In [60]:
df = df.fillna(0)

In [71]:
# features: bed, bath, sqft, lon/lat, garage, age (or year built), and lotsize

features = ['BDRMS', 'FULL_BTH', 'HALF_BTH', 'LIVING_AREA', 'YR_BUILT', 'LAND_SF', 'LAT', 'LONG']
X = df[features] #Features
y = df['AV_TOTAL'] #Target
X

,BDRMS,FULL_BTH,HALF_BTH,LIVING_AREA,YR_BUILT,LAT,LONG
0,2.0,1.0,0.0,1478.0,1900.0,-71.07207,42.355618
1,6.0,3.0,0.0,2202.0,1900.0,-71.03256,42.379173
2,3.0,3.0,0.0,2307.0,1920.0,-71.03255,42.37957
3,5.0,3.0,0.0,2268.0,1905.0,-71.03248,42.379593
4,5.0,3.0,0.0,2028.0,1900.0,-71.03242,42.379616
...,...,...,...,...,...,...,...
101662,1.0,1.0,0.0,620.0,2016.0,-71.10889,42.335903
101663,1.0,1.0,0.0,831.0,2016.0,-71.10889,42.335903
103067,2.0,1.0,0.0,1813.0,2017.0,-71.09162,42.322926
103068,3.0,2.0,0.0,1982.0,2017.0,-71.09162,42.322926


In [72]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [73]:
y

0         2678800
1          541200
2          596100
3          542500
4          518300
           ...   
101662     561500
101663     195300
103067     563900
103068     614800
124193     813500
Name: AV_TOTAL, Length: 87158, dtype: int64

In [74]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [13]:
knn.predict(X_test)

array([1732300,  229700,  360100, ...,  743042, 1263600,  517900])

In [14]:
# X = X_train.iloc[:]

In [15]:
n_neighbors = 6
nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm='ball_tree').fit(X)
# distances, indices = nbrs.kneighbors([[4.0,1.0,1.0,1879.0,1929.0,3304.0,71.053566,42.28941]])
# print(indices)
# print(distances)

In [16]:
# x_slice = X.iloc[0:50]

In [17]:
# Get the neighbors
def add_neighbors(row):
    x = []
    distances, indices = nbrs.kneighbors([row])
    for i in range(n_neighbors):
        x.append(indices[0][i])
    return [x[1]], [x[1], x[2]], [x[1], x[2], x[3]], [x[1], x[2], x[3], x[4]], [x[1], x[2], x[3], x[4], x[5]]

X[["nbrs_1", "nbrs_2", "nbrs_3", "nbrs_4", "nbrs_5"]] = X.apply(add_neighbors, axis=1, result_type="expand")


/Users/kmcdouga/Documents/GitHub/CSCIE-14a/final_project/final_venv/lib/python3.7/site-packages/pandas/core/frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [18]:
col = X[["nbrs_1", "nbrs_2", "nbrs_3", "nbrs_4", "nbrs_5"]]

In [19]:
df_merge = df.join(col, lsuffix='_caller', rsuffix='_other')


In [47]:
for i in df_merge.iterrows():
    print(i[1].PID)
    print(df_merge.iloc[i[1].nbrs_1].PID)

502555000
4236    106985000
Name: PID, dtype: int64
100001000
4277    107035000
Name: PID, dtype: int64
100002000
3021    104840000
Name: PID, dtype: int64
100003000
3257    105216000
Name: PID, dtype: int64
100004000
9305    700145000
Name: PID, dtype: int64
100005000
2564    103762000
Name: PID, dtype: int64
100006000
16332    1202651001
Name: PID, dtype: int64
100007000
31    100031000
Name: PID, dtype: int64
100008000
97    100128000
Name: PID, dtype: int64
100009000
2502    103618000
Name: PID, dtype: int64
100010000
1852    102806000
Name: PID, dtype: int64
100011000
208    100201000
Name: PID, dtype: int64
100012000
3845    106470000
Name: PID, dtype: int64
100013000
2660    103983000
Name: PID, dtype: int64
100014000
3614    106130000
Name: PID, dtype: int64
100015000
1887    102851000
Name: PID, dtype: int64
100016000
50967    2004769000
Name: PID, dtype: int64
100017000
3665    106211000
Name: PID, dtype: int64
100018000
12932    1001085000
Name: PID, dtype: int64
100019000
2

Name: PID, dtype: int64
100195010
8110    602131000
Name: PID, dtype: int64
100195020
7946    601657000
Name: PID, dtype: int64
100197000
2670    104000000
Name: PID, dtype: int64
100198000
17025    1300507000
Name: PID, dtype: int64
100199000
550    100853000
Name: PID, dtype: int64
100200000
20    100020000
Name: PID, dtype: int64
100201000
112    100144000
Name: PID, dtype: int64
100202000
7083    501891000
Name: PID, dtype: int64
100203000
3329    105338000
Name: PID, dtype: int64
100204000
12314    903226000
Name: PID, dtype: int64
100205000
15824    1201382001
Name: PID, dtype: int64
100206000
1682    102603010
Name: PID, dtype: int64
100207000
10751    702698001
Name: PID, dtype: int64
100208000
3349    105446000
Name: PID, dtype: int64
100209000
4072    106753000
Name: PID, dtype: int64
100210000
29908    1702167000
Name: PID, dtype: int64
100211000
88557    503596002
Name: PID, dtype: int64
100212000
519    100814000
Name: PID, dtype: int64
100218000
23739    1502264060
Name: 

Name: PID, dtype: int64
100589000
14923    1103152000
Name: PID, dtype: int64
100590000
354    100541000
Name: PID, dtype: int64
100591000
1971    102952000
Name: PID, dtype: int64
100596000
576    100882000
Name: PID, dtype: int64
100599000
56524    2012051021
Name: PID, dtype: int64
100602000
39948    1809502000
Name: PID, dtype: int64
100603000
9457    700598000
Name: PID, dtype: int64
100604000
52045    2006133000
Name: PID, dtype: int64
100605000
50394    2004153000
Name: PID, dtype: int64
100606000
51008    2004819000
Name: PID, dtype: int64
100613000
25501    1601732000
Name: PID, dtype: int64
100614000
10805    702823000
Name: PID, dtype: int64
100615000
61818    1400487000
Name: PID, dtype: int64
100616000
2259    103300000
Name: PID, dtype: int64
100617000
18664    1303139000
Name: PID, dtype: int64
100618000
11314    703880000
Name: PID, dtype: int64
100619000
45576    1903318000
Name: PID, dtype: int64
100620000
10573    702441000
Name: PID, dtype: int64
100621000
17854    

Name: PID, dtype: int64
100891000
3578    106047000
Name: PID, dtype: int64
100894000
2011    102997000
Name: PID, dtype: int64
100895000
15874    1201607000
Name: PID, dtype: int64
100900000
2739    104295008
Name: PID, dtype: int64
100901000
781    101193000
Name: PID, dtype: int64
100902000
1914    102881000
Name: PID, dtype: int64
100903000
29360    1701364000
Name: PID, dtype: int64
100904000
57848    2102852000
Name: PID, dtype: int64
100905000
44074    1901432000
Name: PID, dtype: int64
100908000
57897    2200052000
Name: PID, dtype: int64
100909000
14494    1102121000
Name: PID, dtype: int64
100911000
304    100373000
Name: PID, dtype: int64
100912000
25681    1601945002
Name: PID, dtype: int64
100913000
67029    203715004
Name: PID, dtype: int64
100914000
1870    102830000
Name: PID, dtype: int64
100919000
11307    703870000
Name: PID, dtype: int64
100920010
3075    104933000
Name: PID, dtype: int64
100921000
13532    1002556000
Name: PID, dtype: int64
100922000
1887    102851

Name: PID, dtype: int64
101193000
589    100901000
Name: PID, dtype: int64
101195000
61364    2205222000
Name: PID, dtype: int64
101196000
17059    1300549000
Name: PID, dtype: int64
101197000
58339    2201170000
Name: PID, dtype: int64
101198000
47332    2000653000
Name: PID, dtype: int64
101199000
18166    1302425000
Name: PID, dtype: int64
101200000
28634    1700456000
Name: PID, dtype: int64
101201000
22021    1405108000
Name: PID, dtype: int64
101202000
30795    1703189000
Name: PID, dtype: int64
101203000
33172    1800775000
Name: PID, dtype: int64
101204000
19259    1400839000
Name: PID, dtype: int64
101205000
17066    1300556000
Name: PID, dtype: int64
101206000
13815    1100516000
Name: PID, dtype: int64
101207000
19890    1401887000
Name: PID, dtype: int64
101208000
28783    1700634000
Name: PID, dtype: int64
101209000
60352    2204039000
Name: PID, dtype: int64
101210000
23025    1501285000
Name: PID, dtype: int64
101211000
799    101212000
Name: PID, dtype: int64
101212000


...

Name: PID, dtype: int64
1804458000
60492    2204245000
Name: PID, dtype: int64
1804459000
51411    2005302000
Name: PID, dtype: int64
1804460000
43888    1901084000
Name: PID, dtype: int64
1804462010
49053    2002589001
Name: PID, dtype: int64
1804466000
39897    1809408000
Name: PID, dtype: int64
1804467000
48583    2002074000
Name: PID, dtype: int64
1804468000
24479    1600282000
Name: PID, dtype: int64
1804469000
19186    1400688000
Name: PID, dtype: int64
1804476010
45136    1902678000
Name: PID, dtype: int64
1804477000
24451    1600196000
Name: PID, dtype: int64
1804485000
40120    1809686000
Name: PID, dtype: int64
1804486000
61476    2205331000
Name: PID, dtype: int64
1804487000
41394    1810903000
Name: PID, dtype: int64
1804488000
54567    2009059000
Name: PID, dtype: int64
1804491000
45376    1903035000
Name: PID, dtype: int64
1804492000
30299    1702655000
Name: PID, dtype: int64
1804494000
8654    603540000
Name: PID, dtype: int64
1804495000
21529    1404481000
Name: PID, d

1804686000
29938    1702203000
Name: PID, dtype: int64
1804688000
58611    2201538000
Name: PID, dtype: int64
1804689000
35904    1804690000
Name: PID, dtype: int64
1804690000
35903    1804689000
Name: PID, dtype: int64
1804691000
47876    2001257000
Name: PID, dtype: int64
1804692000
16239    1202448000
Name: PID, dtype: int64
1804693000
43281    1812946000
Name: PID, dtype: int64
1804694000
48281    2001768000
Name: PID, dtype: int64
1804695000
35983    1804761305
Name: PID, dtype: int64
1804696000
55238    2009892000
Name: PID, dtype: int64
1804697000
18242    1302541000
Name: PID, dtype: int64
1804698000
35913    1804699000
Name: PID, dtype: int64
1804699000
35912    1804698000
Name: PID, dtype: int64
1804700000
56177    2011260000
Name: PID, dtype: int64
1804701000
32997    1800538000
Name: PID, dtype: int64
1804702000
55112    2009750000
Name: PID, dtype: int64
1804706000
58321    2201146000
Name: PID, dtype: int64
1804707000
35627    1804299000
Name: PID, dtype: int64
1804708000

Name: PID, dtype: int64
1804893000
48236    2001722000
Name: PID, dtype: int64
1804894000
55611    2010403000
Name: PID, dtype: int64
1804895000
19493    1401189000
Name: PID, dtype: int64
1804896000
59905    2203433000
Name: PID, dtype: int64
1804897000
44727    1902344000
Name: PID, dtype: int64
1804898000
55566    2010358000
Name: PID, dtype: int64
1804899000
39367    1808594000
Name: PID, dtype: int64
1804900000
42492    1812267027
Name: PID, dtype: int64
1804901000
59473    2202906000
Name: PID, dtype: int64
1804902000
17556    1301522000
Name: PID, dtype: int64
1804903000
48332    2001820000
Name: PID, dtype: int64
1804904000
28210    1605296000
Name: PID, dtype: int64
1804905000
61663    2205580000
Name: PID, dtype: int64
1804907000
40085    1809659000
Name: PID, dtype: int64
1804908000
44728    1902344001
Name: PID, dtype: int64
1804910000
42580    1812269026
Name: PID, dtype: int64
1804912000
31915    1704477000
Name: PID, dtype: int64
1804913000
49160    2002677000
Name: PID,

Name: PID, dtype: int64
1805127000
54489    2008851000
Name: PID, dtype: int64
1805128000
36297    1805127000
Name: PID, dtype: int64
1805129030
14552    1102291000
Name: PID, dtype: int64
1805130000
47035    2000304000
Name: PID, dtype: int64
1805131000
26812    1603506000
Name: PID, dtype: int64
1805133000
31168    1703605000
Name: PID, dtype: int64
1805134000
13214    1001785010
Name: PID, dtype: int64
1805135000
19278    1400865000
Name: PID, dtype: int64
1805136000
43277    1812935000
Name: PID, dtype: int64
1805137000
39662    1808923000
Name: PID, dtype: int64
1805139000
36348    1805202000
Name: PID, dtype: int64
1805141000
36344    1805198000
Name: PID, dtype: int64
1805144000
20701    1403179000
Name: PID, dtype: int64
1805147000
22961    1501202000
Name: PID, dtype: int64
1805148000
36312    1805149000
Name: PID, dtype: int64
1805149000
36313    1805150000
Name: PID, dtype: int64
1805150000
36312    1805149000
Name: PID, dtype: int64
1805151000
36338    1805189000
Name: PID,

KeyboardInterrupt: 

In [86]:
fun = lambda x, y: x + y
fun(1,2)

3